In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2697830516193604329, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 488429122978620538
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:83:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 1374722970411137879
 physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1", name: "/device:GPU:2"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 11704624778281363365
 physical_device_desc: "device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:87:00.0, compute capability: 6.1", name: "/device:GPU:3"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 11643758181486737863
 physical_devic

In [3]:
import keras
import tensorflow as tf
import keras.backend as K
print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.4.0
dim_ordering: tf


Using TensorFlow backend.


In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss

import os


import numpy as np

from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model

from keras.layers import Input, Dense, Activation, Dropout, BatchNormalization


from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

from sklearn.metrics import accuracy_score


import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
output_classes = 4
batch_size = 32
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
epochs = 50

nb_train_samples = 134400
nb_validation_samples = 44800
nb_test_samples = 80

In [23]:
train_dir = 'data/train_patched/'
validation_dir = 'data/validation_patched/'
test_dir = 'data/test_IW/'

In [24]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 134400 images belonging to 4 classes.
Found 44800 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [25]:
def vgg19_model():
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    prediction = Dense(output_classes, activation=tf.nn.softmax)(x)

    model = Model(inputs=base_model.input,outputs=prediction)
    return model

In [26]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001,0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
#          'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
#          'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
#          'decay':hp.choice('decay',[0.0, 0.004, 0.0001, 0.1, 0.3, 0.5]),
          'momentum':hp.choice('momentum',[0.3,0.4,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [27]:
def f_nn(params):   
    print ('Parameters testing: ', params)

    
    sgd_opt=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])

    model = vgg19_model()
    model.compile(optimizer = sgd_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      epochs = 15,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,
      verbose = 1, callbacks=get_callbacks(params))
    
    score = model.evaluate_generator(validation_generator, steps=50)

    print ('Validation Score: ', score[0])
    print ('Validation Accuracy: ',score[1])
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    predicts = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
    predict_class = np.argmax(predicts, axis=1)
    errors = np.where(predict_class != truth)[0]
    print("No of errors = {}/{}".format(len(errors),test_generator.samples))
    
    th= 0.3
    acc = accuracy_score(truth, predict_class > th)
    print("Test Accuracy: {:.4f}".format(acc*100))
    print("*_*" * 50)

    sys.stdout.flush() 
    
    return {'loss': score[0], 'status': STATUS_OK, 'model': model}

In [ ]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=5, verbose=1)]
    return callbacks

In [ ]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'lr': 1e-05, 'momentum': 0.5, 'nesterov': False}
Epoch 1/15
4200/4200 [==============================] - 5342s 1s/step - loss: 1.5735 - acc: 0.4314 - val_loss: 1.2148 - val_acc: 0.5644
Epoch 2/15
4199/4200 [============================>.] - ETA: 0s - loss: 1.1671 - acc: 0.6032